In [1]:
import pandas as pd
import glob
from tqdm.notebook import tqdm

In [2]:
logs = [('Server_' + x.split('_')[-1].replace('.csv', ''), x) for x in glob.glob("../logs/*.csv")]
columns = ['chunk_x', 'chunk_y', 'version', 'producer', 'produced_at']

chunk_produced_map={}
chunk_arrvied_at_server_map={}
for (server, file) in logs:
    columns.append("arrived_at_" + server)
    columns.append("sync_latency_" + server)
    chunk_arrvied_at_server_map[server]={}
syncLatencies = pd.DataFrame(columns=columns)

In [3]:
for (server, file) in logs:
    log = pd.read_csv(file, sep='\t', header=None, names=["time", "type", "x", "y", "v"])
    print("Processing CSV file of server " + server)
    
    for index, row in tqdm(log.iterrows(),total=len(log)):
        if row['type'] == 'OUT':
            chunk_produced_map[str(row['x']) + '_' + str(row['y']) + '_' + str(row['v'])] = (int(row['time']), server)
        elif row['type'] == 'IN':
            chunk_arrvied_at_server_map[server][str(row['x']) + '_' + str(row['y']) + '_' + str(row['v'])] = int(row['time'])

Processing CSV file of server Server_2



Processing CSV file of server Server_3



Processing CSV file of server Server_1



Processing CSV file of server Server_0


In [4]:
# clear the dataframe
syncLatencies = syncLatencies.iloc[0:0]

for chunk, (timestamp, server) in tqdm(chunk_produced_map.items(), total=len(chunk_produced_map)):
    cd = chunk.split('_')
    x = cd[0]
    y = cd[1]
    v = cd[2]
    entry = {'chunk_x':  x,
             'chunk_y': y,
             'version': v,
             'producer': server,
             'produced_at': timestamp}
    for (server, file) in logs:
        if chunk in chunk_arrvied_at_server_map[server]:
            arrived_time = chunk_arrvied_at_server_map[server][chunk]
            sync_latency = arrived_time - timestamp
            entry["arrived_at_"+server] = arrived_time
            entry["sync_latency_"+server] = sync_latency

    syncLatencies = syncLatencies.append(entry, ignore_index=True)
syncLatencies

KeyboardInterrupt: 

In [11]:
syncLatencies.mean()

chunk_x                  3.218474e+04
chunk_y                  3.338962e+04
version                  1.739951e+02
produced_at              1.580829e+12
arrived_at_Server_2      1.580829e+12
sync_latency_Server_2    6.408893e+02
arrived_at_Server_3      1.580829e+12
sync_latency_Server_3    3.496588e+02
arrived_at_Server_1      1.580829e+12
sync_latency_Server_1    3.393989e+02
arrived_at_Server_0      1.580829e+12
sync_latency_Server_0    3.759239e+02
dtype: float64